**Importaciónn de los paquetes y módulos necesarios**

In [40]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

**Carga, visualización y limpieza de datos**

In [2]:
df = pd.read_csv("./data/Clasificaciones_LaLiga.csv")

In [3]:
#Mostramos las primeras 10 filas del DF para conocer su contenido
df.head(10)

,Temporada,Posicion J38,Equipo,Puntos J38,PJ Total,G Total,E Total,PTotal,GF Total,GC Total,...,Posicion J29,Equipo J29,Puntos J29,PJ 29,G 29,E 29,P 29,GF 29,GC 29,Dif 29
0,1999,1,Barcelona,74.0,38.0,23.0,5.0,10.0,78.0,56.0,...,1,Barcelona,59,29,18,5,6,65,33,32
1,1999,2,Athletic Club,65.0,38.0,17.0,14.0,7.0,52.0,42.0,...,2,R. C. Celta de Vigo,52,29,14,10,5,58,29,29
2,1999,3,Real Sociedad,63.0,38.0,16.0,15.0,7.0,60.0,37.0,...,3,Valencia C. F.,51,29,15,6,8,44,28,16
3,1999,4,Real Madrid C. F.,63.0,38.0,17.0,12.0,9.0,63.0,45.0,...,4,R. C. D. Mallorca,51,29,15,6,8,32,21,11
4,1999,5,R. C. D. Mallorca,60.0,38.0,16.0,12.0,10.0,55.0,39.0,...,5,Real Madrid C. F.,49,29,15,4,10,58,49,9
5,1999,6,R. C. Celta de Vigo,60.0,38.0,17.0,9.0,12.0,54.0,47.0,...,6,Deportivo de La Coruña,48,29,13,9,7,41,31,10
6,1999,7,Atlético de Madrid,60.0,38.0,16.0,12.0,10.0,79.0,56.0,...,7,Athletic Club,44,29,12,8,9,33,34,-1
7,1999,8,Real Betis Balompié,59.0,38.0,17.0,8.0,13.0,49.0,50.0,...,8,Real Sociedad,42,29,11,9,9,35,32,3
8,1999,9,Valencia C. F.,55.0,38.0,16.0,7.0,15.0,58.0,52.0,...,9,Real Zaragoza,40,29,11,7,11,41,39,2
9,1999,10,R. C. D. Espanyol,53.0,38.0,12.0,17.0,9.0,44.0,31.0,...,10,R. C. D. Espanyol,39,29,9,12,8,31,33,-2


In [4]:
#Obtenemos la información de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Temporada     520 non-null    int64  
 1   Posicion J38  520 non-null    int64  
 2   Equipo        500 non-null    object 
 3   Puntos J38    500 non-null    float64
 4   PJ Total      500 non-null    float64
 5   G Total       500 non-null    float64
 6   E Total       500 non-null    float64
 7   PTotal        500 non-null    float64
 8   GF Total      500 non-null    float64
 9   GC Total      500 non-null    float64
 10  Dif Total     500 non-null    float64
 11  Posicion J29  520 non-null    int64  
 12  Equipo J29    520 non-null    object 
 13  Puntos J29    520 non-null    int64  
 14  PJ 29         520 non-null    int64  
 15  G 29          520 non-null    int64  
 16  E 29          520 non-null    int64  
 17  P 29          520 non-null    int64  
 18  GF 29         520 non-null    

In [5]:
print("Estadisticas descriptivas para columnas numericas: ")
df.describe()

Estadisticas descriptivas para columnas numericas: 


,Temporada,Posicion J38,Puntos J38,PJ Total,G Total,E Total,PTotal,GF Total,GC Total,Dif Total,Posicion J29,Puntos J29,PJ 29,G 29,E 29,P 29,GF 29,GC 29,Dif 29
count,520.000000,520.000000,500.000000,500.0,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,520.000000,520.000000,520.000000,520.000000,520.000000,520.000000,520.000000,520.000000,520.000000
mean,2011.500000,10.500000,52.162000,38.0,14.170000,9.660000,14.170000,50.630000,50.610000,0.020000,10.500000,39.748077,28.996154,10.759615,7.476923,10.759615,38.248077,38.248077,0.000000
std,7.507222,5.771834,15.392518,0.0,5.511611,2.872089,4.860076,16.406079,11.562928,24.058704,5.771834,12.437590,0.061958,4.495117,2.523769,3.912076,12.942514,9.480636,19.151862
min,1999.000000,1.000000,13.000000,38.0,2.000000,1.000000,1.000000,22.000000,18.000000,-54.000000,1.000000,11.000000,28.000000,1.000000,0.000000,0.000000,16.000000,9.000000,-40.000000
25%,2005.000000,5.750000,42.000000,38.0,10.000000,8.000000,11.000000,40.000000,43.000000,-16.000000,5.750000,31.000000,29.000000,8.000000,6.000000,8.000000,29.750000,32.000000,-13.000000
50%,2011.500000,10.500000,49.000000,38.0,13.000000,10.000000,15.000000,47.000000,51.000000,-5.000000,10.500000,37.000000,29.000000,10.000000,7.000000,11.000000,35.000000,39.000000,-4.000000
75%,2018.000000,15.250000,60.000000,38.0,17.000000,12.000000,17.000000,57.000000,58.000000,11.000000,15.250000,46.000000,29.000000,13.000000,9.000000,13.000000,43.000000,44.000000,9.000000
max,2024.000000,20.000000,100.000000,38.0,32.000000,18.000000,29.000000,121.000000,94.000000,89.000000,20.000000,78.000000,29.000000,25.000000,16.000000,21.000000,93.000000,69.000000,68.000000


In [6]:
#Verificación de valores faltantes
print(f"Valores faltantes por columna:")
print(df.isnull().sum())

Valores faltantes por columna:
Temporada        0
Posicion J38     0
Equipo          20
Puntos J38      20
PJ Total        20
G Total         20
E Total         20
PTotal          20
GF Total        20
GC Total        20
Dif Total       20
Posicion J29     0
Equipo J29       0
Puntos J29       0
PJ 29            0
G 29             0
E 29             0
P 29             0
GF 29            0
GC 29            0
Dif 29           0
dtype: int64


In [7]:
columnas_nulos = ["Puntos J38", "PJ Total","G Total", "E Total", "PTotal", "GF Total", "GC Total", "Dif Total"]
imputer = KNNImputer()

In [8]:
#Imputamos los valores faltantes 
df_2= df.copy()
df_2[columnas_nulos]= imputer.fit_transform(df_2[columnas_nulos])

In [9]:
# Imputar valores faltantes en la columna "Equipo" con "Desconocido"
df_2['Equipo'].fillna('Desconocido', inplace=True)

In [10]:
# Verificar si hay valores faltantes después de la imputación
print("Valores faltantes después de la imputación:")
print(df_2.isnull().sum())

Valores faltantes después de la imputación:
Temporada       0
Posicion J38    0
Equipo          0
Puntos J38      0
PJ Total        0
G Total         0
E Total         0
PTotal          0
GF Total        0
GC Total        0
Dif Total       0
Posicion J29    0
Equipo J29      0
Puntos J29      0
PJ 29           0
G 29            0
E 29            0
P 29            0
GF 29           0
GC 29           0
Dif 29          0
dtype: int64


In [11]:
# Codificar variables categóricas usando one-hot encoding
df_encoded = pd.get_dummies(df_2, columns=['Equipo', 'Equipo J29'], dtype=int)

In [12]:
#Vemos que se han realizado las modificaciones de forma correcta y vemos como se ha quedado el DF
df_encoded.head()

,Temporada,Posicion J38,Puntos J38,PJ Total,G Total,E Total,PTotal,GF Total,GC Total,Dif Total,...,Equipo J29_S. D. Huesca,Equipo J29_Sevilla F. C.,Equipo J29_Sporting de Gijón,Equipo J29_Tenerife,Equipo J29_U. D. Almería,Equipo J29_U. D. Las Palmas,Equipo J29_UD Salamanca,Equipo J29_Valencia C. F.,Equipo J29_Villarreal C. F.,Equipo J29_Xerez
0,1999,1,74.0,38.0,23.0,5.0,10.0,78.0,56.0,22.0,...,0,0,0,0,0,0,0,0,0,0
1,1999,2,65.0,38.0,17.0,14.0,7.0,52.0,42.0,10.0,...,0,0,0,0,0,0,0,0,0,0
2,1999,3,63.0,38.0,16.0,15.0,7.0,60.0,37.0,23.0,...,0,0,0,0,0,0,0,1,0,0
3,1999,4,63.0,38.0,17.0,12.0,9.0,63.0,45.0,18.0,...,0,0,0,0,0,0,0,0,0,0
4,1999,5,60.0,38.0,16.0,12.0,10.0,55.0,39.0,16.0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Definir las características y la variable objetivo
X = df_encoded.drop(columns=['Puntos J38'])  # Características
y = df_encoded['Puntos J38']  # Variable objetivo

In [14]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Mostrar las dimensiones de los conjuntos de datos de entrenamiento y prueba
print("Dimensiones de los conjuntos de datos:")
print("Conjunto de entrenamiento - Características:", X_train.shape)
print("Conjunto de entrenamiento - Etiquetas:", y_train.shape)
print("Conjunto de prueba - Características:", X_test.shape)
print("Conjunto de prueba - Etiquetas:", y_test.shape)

Dimensiones de los conjuntos de datos:
Conjunto de entrenamiento - Características: (416, 126)
Conjunto de entrenamiento - Etiquetas: (416,)
Conjunto de prueba - Características: (104, 126)
Conjunto de prueba - Etiquetas: (104,)


**Modelado**

In [16]:
# Inicializar el modelo de regresión lineal
modelo = LinearRegression()

In [17]:
# Entrenar el modelo con los datos de entrenamiento
modelo.fit(X_train, y_train)

LinearRegression()

In [18]:
# Predecir los puntos para el conjunto de entrenamiento y prueba
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

In [19]:
# CalcuCalculamos el error cuadrático medio (MSE) en el conjunto de entrenamiento y prueba
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

In [20]:
# Mostramos el MSE en el conjunto de entrenamiento y prueba
print("Error cuadrático medio (MSE) en el conjunto de entrenamiento:", mse_train)
print("Error cuadrático medio (MSE) en el conjunto de prueba:", mse_test)

Error cuadrático medio (MSE) en el conjunto de entrenamiento: 0.0019029461474605264
Error cuadrático medio (MSE) en el conjunto de prueba: 18984429548622.316


Prueba regresión Ridge

In [21]:
# Inicializar el modelo de regresión Ridge
ridge_model = Ridge(alpha=1.0)  # Alpha es el parámetro de regularización

In [22]:
# Entrenar el modelo de regresión Ridge con los datos de entrenamiento
ridge_model.fit(X_train, y_train)

Ridge()

In [23]:
# Predecir los puntos para el conjunto de entrenamiento y prueba
y_train_pred_ridge = ridge_model.predict(X_train)
y_test_pred_ridge = ridge_model.predict(X_test)



In [24]:
# Calcular el error cuadrático medio (MSE) en el conjunto de entrenamiento y prueba
mse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge)
mse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge)

In [25]:
# Mostrar el MSE en el conjunto de entrenamiento y prueba
print("Ridge Regression:")
print("Error cuadrático medio (MSE) en el conjunto de entrenamiento:", mse_train_ridge)
print("Error cuadrático medio (MSE) en el conjunto de prueba:", mse_test_ridge)

Ridge Regression:
Error cuadrático medio (MSE) en el conjunto de entrenamiento: 0.001930237752733867
Error cuadrático medio (MSE) en el conjunto de prueba: 0.0876376612594871


In [26]:
# Definir los valores de alpha que se probarán
alphas = [0.1, 0.5, 1.0, 5.0, 10.0]



In [27]:
# Crear un diccionario con los parámetros que se probarán
param_grid = {'alpha': alphas}



In [28]:
# Inicializar el modelo de regresión Ridge
ridge_model = Ridge()



In [29]:
# Inicializar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')



In [30]:
# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]},
             scoring='neg_mean_squared_error')

In [31]:
# Mostrar los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print("Mejor puntaje (neg_mean_squared_error):", grid_search.best_score_)

Mejores parámetros encontrados:
{'alpha': 1.0}
Mejor puntaje (neg_mean_squared_error): -0.003020598853875057


In [32]:
# Inicializar el modelo de regresión Ridge con el mejor valor de alpha encontrado
best_alpha = grid_search.best_params_['alpha']
ridge_model_best = Ridge(alpha=best_alpha)

# Entrenar el modelo de regresión Ridge con los datos de entrenamiento
ridge_model_best.fit(X_train, y_train)

# Predecir los puntos para el conjunto de entrenamiento y prueba
y_train_pred_ridge_best = ridge_model_best.predict(X_train)
y_test_pred_ridge_best = ridge_model_best.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de entrenamiento y prueba
mse_train_ridge_best = mean_squared_error(y_train, y_train_pred_ridge_best)
mse_test_ridge_best = mean_squared_error(y_test, y_test_pred_ridge_best)

# Mostrar el MSE en el conjunto de entrenamiento y prueba con el mejor modelo
print("Ridge Regression con el mejor valor de alpha:")
print("Error cuadrático medio (MSE) en el conjunto de entrenamiento:", mse_train_ridge_best)
print("Error cuadrático medio (MSE) en el conjunto de prueba:", mse_test_ridge_best)

Ridge Regression con el mejor valor de alpha:
Error cuadrático medio (MSE) en el conjunto de entrenamiento: 0.001930237752733867
Error cuadrático medio (MSE) en el conjunto de prueba: 0.0876376612594871


RandomForest


In [34]:
# Iniciamos el modelo de Random Forest
random_forest_model = RandomForestRegressor(random_state=42)

In [35]:
# Entrenamos el modelo con los datos de entrenamiento
random_forest_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [36]:
# Predecir los puntos para el conjunto de entrenamiento y prueba
y_train_pred_rf = random_forest_model.predict(X_train)
y_test_pred_rf = random_forest_model.predict(X_test)

In [37]:
# Calcular el error cuadrático medio (MSE) en el conjunto de entrenamiento y prueba
mse_train_rf = mean_squared_error(y_train, y_train_pred_rf)
mse_test_rf = mean_squared_error(y_test, y_test_pred_rf)


In [38]:
# Mostrar el MSE en el conjunto de entrenamiento y prueba
print("Random Forest Regression:")
print("Error cuadrático medio (MSE) en el conjunto de entrenamiento:", mse_train_rf)
print("Error cuadrático medio (MSE) en el conjunto de prueba:", mse_test_rf)

Random Forest Regression:
Error cuadrático medio (MSE) en el conjunto de entrenamiento: 0.2471731046894231
Error cuadrático medio (MSE) en el conjunto de prueba: 1.2722917023884615


In [42]:
# Definir la cuadrícula de hiperparámetros a explorar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [43]:
# Inicializar el modelo de Random Forest
random_forest_model = RandomForestRegressor(random_state=42)

In [44]:
# Inicializar la búsqueda de hiperparámetros con validación cruzada
random_search = RandomizedSearchCV(estimator=random_forest_model, param_distributions=param_grid, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [45]:
# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
random_search.fit(X_train, y_train)

c:\Users\Adri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
200 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Adri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Adri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\Adri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py"

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='neg_mean_squared_error')

In [46]:
# Mostrar los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:")
print(random_search.best_params_)
print("Mejor puntaje (neg_mean_squared_error):", random_search.best_score_)

Mejores parámetros encontrados:
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}
Mejor puntaje (neg_mean_squared_error): -5.436739788885431


In [47]:
# Utilizar los mejores parámetros encontrados para construir un nuevo modelo de Random Forest
best_params = random_search.best_params_
random_forest_model_best = RandomForestRegressor(**best_params, random_state=42)

In [48]:
# Entrenar el modelo de Random Forest con los datos de entrenamiento
random_forest_model_best.fit(X_train, y_train)

RandomForestRegressor(max_depth=30, max_features='sqrt', n_estimators=300,
                      random_state=42)

In [49]:
# Predecir los puntos para el conjunto de entrenamiento y prueba
y_train_pred_rf_best = random_forest_model_best.predict(X_train)
y_test_pred_rf_best = random_forest_model_best.predict(X_test)

In [50]:
# Calcular el error cuadrático medio (MSE) en el conjunto de entrenamiento y prueba
mse_train_rf_best = mean_squared_error(y_train, y_train_pred_rf_best)
mse_test_rf_best = mean_squared_error(y_test, y_test_pred_rf_best)

In [51]:
# Mostrar el MSE en el conjunto de entrenamiento y prueba con el mejor modelo
print("Random Forest Regression con los mejores hiperparámetros:")
print("Error cuadrático medio (MSE) en el conjunto de entrenamiento:", mse_train_rf_best)
print("Error cuadrático medio (MSE) en el conjunto de prueba:", mse_test_rf_best)

Random Forest Regression con los mejores hiperparámetros:
Error cuadrático medio (MSE) en el conjunto de entrenamiento: 0.694028894574395
Error cuadrático medio (MSE) en el conjunto de prueba: 3.871101451544556


In [52]:
# Definir la cuadrícula de hiperparámetros a explorar
param_grid_regularized = {
    'n_estimators': [best_params['n_estimators']],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt']
}

In [53]:
# Inicializar la búsqueda de hiperparámetros con validación cruzada
random_search_regularized = RandomizedSearchCV(estimator=random_forest_model, param_distributions=param_grid_regularized, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [54]:
# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
random_search_regularized.fit(X_train, y_train)

c:\Users\Adri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 27 is smaller than n_iter=100. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100,
                   param_distributions={'max_depth': [10, 20, 30],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [300]},
                   random_state=42, scoring='neg_mean_squared_error')

In [55]:
# Mostrar los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados con regularización:")
print(random_search_regularized.best_params_)
print("Mejor puntaje (neg_mean_squared_error) con regularización:", random_search_regularized.best_score_)

Mejores parámetros encontrados con regularización:
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Mejor puntaje (neg_mean_squared_error) con regularización: -5.392479996900138


In [56]:
# Utilizar los mejores parámetros encontrados con regularización para construir un nuevo modelo de Random Forest
best_params_regularized = random_search_regularized.best_params_
random_forest_model_regularized = RandomForestRegressor(**best_params_regularized, random_state=42)

In [57]:
# Entrenar el modelo de Random Forest regularizado con los datos de entrenamiento
random_forest_model_regularized.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=300,
                      random_state=42)

In [58]:
# Predecir los puntos para el conjunto de entrenamiento y prueba con el modelo regularizado
y_train_pred_rf_regularized = random_forest_model_regularized.predict(X_train)
y_test_pred_rf_regularized = random_forest_model_regularized.predict(X_test)

In [59]:
# Calcular el error cuadrático medio (MSE) en el conjunto de entrenamiento y prueba con el modelo regularizado
mse_train_rf_regularized = mean_squared_error(y_train, y_train_pred_rf_regularized)
mse_test_rf_regularized = mean_squared_error(y_test, y_test_pred_rf_regularized)

In [60]:
# Mostrar el MSE en el conjunto de entrenamiento y prueba con el modelo regularizado
print("Random Forest Regression con los mejores hiperparámetros encontrados con regularización:")
print("Error cuadrático medio (MSE) en el conjunto de entrenamiento:", mse_train_rf_regularized)
print("Error cuadrático medio (MSE) en el conjunto de prueba:", mse_test_rf_regularized)

Random Forest Regression con los mejores hiperparámetros encontrados con regularización:
Error cuadrático medio (MSE) en el conjunto de entrenamiento: 0.7010261007984055
Error cuadrático medio (MSE) en el conjunto de prueba: 3.7776038407663646


In [61]:
from sklearn.ensemble import RandomForestRegressor
import joblib

# Utilizar todos los datos disponibles para el entrenamiento final
X_final = X.values  
y_final = y.values  


In [64]:
# Definir el modelo de Random Forest con los mejores hiperparámetros encontrados
random_forest_model_final = RandomForestRegressor(**best_params_regularized, random_state=42)

In [65]:
# Entrenar el modelo de Random Forest con todos los datos disponibles
random_forest_model_final.fit(X_final, y_final)

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=300,
                      random_state=42)

Guardado del modelo

In [66]:
# Guardar el modelo entrenado en un archivo
joblib.dump(random_forest_model_final, 'random_forest_model_final.pkl')

['random_forest_model_final.pkl']

Prueba del modelo

In [67]:
# Cargar el modelo entrenado desde el archivo
random_forest_model_final = joblib.load('random_forest_model_final.pkl')

In [68]:
df_prediccion = pd.read_csv("./data/Clasificaciones_LaLiga.csv")

In [69]:
columnas_nulos_2 = ["Puntos J38", "PJ Total","G Total", "E Total", "PTotal", "GF Total", "GC Total", "Dif Total"]

In [70]:
#Imputamos los valores faltantes 
df_prediccion_2= df_prediccion.copy()
df_prediccion_2[columnas_nulos_2]= imputer.fit_transform(df_prediccion_2[columnas_nulos_2])

In [71]:
# Imputar valores faltantes en la columna "Equipo" con "Desconocido"
df_prediccion_2['Equipo'].fillna('Desconocido', inplace=True)

In [72]:
# Codificar variables categóricas usando one-hot encoding
df_encoded_2 = pd.get_dummies(df_prediccion_2, columns=['Equipo', 'Equipo J29'], dtype=int)

In [73]:
df_encoded_2

,Temporada,Posicion J38,Puntos J38,PJ Total,G Total,E Total,PTotal,GF Total,GC Total,Dif Total,...,Equipo J29_S. D. Huesca,Equipo J29_Sevilla F. C.,Equipo J29_Sporting de Gijón,Equipo J29_Tenerife,Equipo J29_U. D. Almería,Equipo J29_U. D. Las Palmas,Equipo J29_UD Salamanca,Equipo J29_Valencia C. F.,Equipo J29_Villarreal C. F.,Equipo J29_Xerez
0,1999,1,74.000,38.0,23.00,5.00,10.00,78.00,56.00,22.00,...,0,0,0,0,0,0,0,0,0,0
1,1999,2,65.000,38.0,17.00,14.00,7.00,52.00,42.00,10.00,...,0,0,0,0,0,0,0,0,0,0
2,1999,3,63.000,38.0,16.00,15.00,7.00,60.00,37.00,23.00,...,0,0,0,0,0,0,0,1,0,0
3,1999,4,63.000,38.0,17.00,12.00,9.00,63.00,45.00,18.00,...,0,0,0,0,0,0,0,0,0,0
4,1999,5,60.000,38.0,16.00,12.00,10.00,55.00,39.00,16.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,2024,16,52.162,38.0,14.17,9.66,14.17,50.63,50.61,0.02,...,0,1,0,0,0,0,0,0,0,0
516,2024,17,52.162,38.0,14.17,9.66,14.17,50.63,50.61,0.02,...,0,0,0,0,0,0,0,0,0,0
517,2024,18,52.162,38.0,14.17,9.66,14.17,50.63,50.61,0.02,...,0,0,0,0,0,0,0,0,0,0
518,2024,19,52.162,38.0,14.17,9.66,14.17,50.63,50.61,0.02,...,0,0,0,0,0,0,0,0,0,0


In [74]:
X_prediccion = df_encoded_2.drop(columns=['Puntos J38'])

In [75]:
X_prediccion.head()

,Temporada,Posicion J38,PJ Total,G Total,E Total,PTotal,GF Total,GC Total,Dif Total,Posicion J29,...,Equipo J29_S. D. Huesca,Equipo J29_Sevilla F. C.,Equipo J29_Sporting de Gijón,Equipo J29_Tenerife,Equipo J29_U. D. Almería,Equipo J29_U. D. Las Palmas,Equipo J29_UD Salamanca,Equipo J29_Valencia C. F.,Equipo J29_Villarreal C. F.,Equipo J29_Xerez
0,1999,1,38.0,23.0,5.0,10.0,78.0,56.0,22.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1999,2,38.0,17.0,14.0,7.0,52.0,42.0,10.0,2,...,0,0,0,0,0,0,0,0,0,0
2,1999,3,38.0,16.0,15.0,7.0,60.0,37.0,23.0,3,...,0,0,0,0,0,0,0,1,0,0
3,1999,4,38.0,17.0,12.0,9.0,63.0,45.0,18.0,4,...,0,0,0,0,0,0,0,0,0,0
4,1999,5,38.0,16.0,12.0,10.0,55.0,39.0,16.0,5,...,0,0,0,0,0,0,0,0,0,0


In [90]:
# Paso adicional: Eliminar los nombres de las características de los datos de entrada
X_prediccion_no_names =X_prediccion.values

In [91]:
X_prediccion_no_names

array([[1.999e+03, 1.000e+00, 3.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.999e+03, 2.000e+00, 3.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.999e+03, 3.000e+00, 3.800e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.024e+03, 1.800e+01, 3.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.024e+03, 1.900e+01, 3.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.024e+03, 2.000e+01, 3.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [92]:
predicciones = random_forest_model_final.predict(X_prediccion)

c:\Users\Adri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [93]:
# Utiliza el modelo cargado para hacer predicciones en los datos preprocesados
predicciones = random_forest_model_final.predict(X_prediccion_no_names)
print(predicciones)

[74.67333333 65.79162    64.13666667 63.72333333 61.04       60.23720667
 59.94       57.67441333 54.12327111 52.90441333 49.3988084  48.60058025
 47.8788847  45.26123048 45.51208785 44.33527399 43.09362135 39.09047619
 37.61792095 18.19666667 69.32333333 65.09441333 64.10333333 63.53
 62.03054    60.20012333 53.61384222 52.17054    51.41162    51.13617467
 49.432      48.16703333 46.77155072 46.93763192 45.81461667 44.2213851
 44.33367875 41.88488889 38.9        30.99108    79.82666667 72.33
 69.92666667 64.11441333 63.32387333 58.46054    57.05333333 55.91387333
 50.63347467 49.61424333 46.19608    43.88475084 43.98004135 43.0406746
 42.17311111 42.41887333 41.59569407 40.58909556 38.80733333 37.77041667
 73.59       68.55666667 66.37666667 64.24       60.72720667 58.82120333
 54.78162    53.84182074 52.45608    51.93044667 49.0082963  47.81020963
 46.66248565 46.19381259 43.72433835 43.32511111 42.08084921 40.35166667
 38.38333333 36.93190476 78.40333333 74.68333333 71.46054    61.6

In [96]:
import pandas as pd

# Crear DataFrame con los puntos predichos
df_resultados = pd.DataFrame({'Puntos_Predichos': predicciones})

# Ordenar DataFrame en orden descendente según los puntos predichos
df_resultados = df_resultados.sort_values(by='Puntos_Predichos', ascending=False)

# Agregar la columna de clasificación
df_resultados['Clasificacion'] = range(1, len(df_resultados) + 1)

# Limitar el DataFrame a los primeros veinte equipos
df_resultados = df_resultados.head
# Mostrar los resultados
print(df_resultados)

<bound method NDFrame.head of      Puntos_Predichos  Clasificacion
260         97.490000              1
280         96.626667              2
220         96.603333              3
240         95.303333              4
221         94.618889              5
..                ...            ...
398         23.420000            516
339         22.276667            517
399         22.097747            518
379         21.677747            519
19          18.196667            520

[520 rows x 2 columns]>
